### Setup Model

In [ ]:
!pip install git+https://github.com/huggingface/transformers torch accelerate bitsandbytes langchain

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import subprocess

command = "pip install -U sentence-transformers chromadb"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [ ]:
import transformers

from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
)
# Download the Mistral 7B Instruct Model and Tokenizer
model_name = "bn22/Mistral-7B-Instruct-v0.1-sharded"
model = AutoModelForCausalLM.from_pretrained(model_name,load_in_4bit=True,torch_dtype=torch.bfloat16,quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Integrating the Model with LangChain


In [ ]:
# import transformers
#Create Text Generation Pipeline
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    # "temperature" is a hyperparameter that controls the randomness of the generated text. It modulates the probability distribution of the next token predicted by the model.
    temperature = 0.2,
    repetition_penalty = 1.1,
    # To generate whole text at once in single string format
    return_full_text=True,
    max_new_tokens=1000,
)

In [ ]:
#Creating an LLM instance
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
# Downloading embedding model
embedding_model = SentenceTransformerEmbeddings(model_name='BAAI/bge-large-zh-v1.5')

### Reading dataset

In [ ]:
# !wget train.txt https://github.com/Nikshay-Jain/dostE/blob/22c3c68535828d94093b42c499cafbf5f9bdec8e/train.txt

In [ ]:
# with open('/content/train.txt', 'r') as file:
#     file_contents = file.read()

In [ ]:
# file_contents

In [ ]:
# import json

# raw = file_contents
# parsed_data = json.loads(raw)
# items = parsed_data['payload']['blob']['rawLines']

In [ ]:
# data = ""
# separator = '\n'
# for i in items:
#   data+=(" "+i)

In [ ]:
# print(data)

 Introduction = """ Core Values To be an academic institution in dynamic equilibrium with its social, ecological, and economic environment striving continuously for excellence in the fields of education, research, and technological service to the nation.  In pursuit of its mission, IITM will: 1. Develop human resources to serve the nation. 2. Recognize teaching as a unifying activity. 3. Nurture integrity, creativity, and academic freedom. 4. Retain a willingness to experiment with new paradigms  Vision: To be an academic institution in dynamic equilibrium with its social, ecological, and economic environment striving continuously for excellence in the fields of education, research, and technological service to the nation.  Mission: 1. To create and sustain a community of learning in which students acquire knowledge and learn to apply it professionally with due consideration for ethical, ecological, and economic issues. 2. To pursue research and disseminate research findings. 3. To pro

In [ ]:
# data_dict = dict(zip(titles, data_values))

In [ ]:
# print(data_dict)

{'Introduction': ' Core Values To be an academic institution in dynamic equilibrium with its social, ecological, and economic environment striving continuously for excellence in the fields of education, research, and technological service to the nation.  In pursuit of its mission, IITM will: 1. Develop human resources to serve the nation. 2. Recognize teaching as a unifying activity. 3. Nurture integrity, creativity, and academic freedom. 4. Retain a willingness to experiment with new paradigms  Vision: To be an academic institution in dynamic equilibrium with its social, ecological, and economic environment striving continuously for excellence in the fields of education, research, and technological service to the nation.  Mission: 1. To create and sustain a community of learning in which students acquire knowledge and learn to apply it professionally with due consideration for ethical, ecological, and economic issues. 2. To pursue research and disseminate research findings. 3. To prov

In [ ]:
Intro = """
IIT Madras or IITM or IIT M, is consistently ranked as the #1 University by NIRF for 7 consecutive years in Engineering segment and 5 years in overall category.

Core Values
To be an academic institution in dynamic equilibrium with its social, ecological, and economic environment striving continuously for excellence in the fields of education, research, and technological service to the nation.

In pursuit of its mission, IITM will:
1. Develop human resources to serve the nation.
2. Recognize teaching as a unifying activity.
3. Nurture integrity, creativity, and academic freedom.
4. Retain a willingness to experiment with new paradigms

Vision:
To be an academic institution in dynamic equilibrium with its social, ecological, and economic environment striving continuously for excellence in the fields of education, research, and technological service to the nation.

Mission:
1. To create and sustain a community of learning in which students acquire knowledge and learn to apply it professionally with due consideration for ethical, ecological, and economic issues.
2. To pursue research and disseminate research findings.
3. To provide knowledge-based technological services to satisfy the needs of society and the industry.
4. To help in building national capabilities in science, technology, humanities, management, education, and research.
"""

TechSoc = """

The Institute technical Society, better known as TechSoc has an amazing ideology operating ints backend. In 2007, a small group of highly ardent IIT Madras students formed a team to inspire the creation of competent technology solutions to solve society's most pressing problems through a set of competitions and events.

The innovative spirit and the disruptive drive that often characterizes the TechSoc culture at IIT Madras. TechSoc embodies an environment where the unconventional, the outliers, and those challenging norms feel at home!

It's a community welcoming those who see the world differently, the ones pushing against the established rules, and who believe that change is achievable. It echoes the belief that those deemed 'crazy' enough to aim for change are often the very ones making a tangible impact, embodying the core ethos of TechSoc – encouraging out-of-the-box thinking and the pursuit of groundbreaking technological solutions.

BuildSchool:
It's a 10 week long contest where 10 teams contest for 1 prize for the prototype they build!
The process starts from the application phase which happens in the start of each semester where team of students submit their project ideas to the BuildSchool teams which is then evaluated on various parameters about the practicality and applicability of the idea in real life. Then the top 10 teams are shortlisted to for actually building the product under the supervision and support of the BuildSchool team

IHC: Inter hostel competitions (IHC) is a vertical under techsoc. It oversees all the techsoc competitions all throughout the semester and at the end of the semester it declares the winner of the semester long competition by calculating the total points won by the hostels. IHC is formed keeping in mind that people should be engaged in semester long technical, quantitative, finance and trading events. Major events that happened during the semesters are: Analytics hackathon,Skycraft, Photobomb, IoT automate your room,Scener, Mathletics, Algo trading, 24 hours webops hackathon, botstacle, manual robotics, guestimate premier league and yojana.
This vertical is headed by mainly 2 cores in techsoc and overall it hires 6-7 managers.

Generally in the application process you are asked to come up with a competition during the semester and ideate on it. Also with this you would be put in situations where you will be asked how will you be able to conduct the competitions during the quiz week, how many participants are you expecting and how would you ensure the smooth running of the competition. If some discrepancies happens like people arguing for points how would you deal with that?

IHC is an overall managerial por which requires a deep level of understanding of the logistics of the competitions you are coming up with. It asks you to manage and look after those competitions so that it happens smoothie. You will have to create notion documents to write down about the logistics of the competition, create google forms, create google docs, maintain a spreadsheet of the participants in the competition.
At the end of the tenure you will walk out with a certain sense of managerial responsibility and exposure which would be helpful to you in ideating and conducting any future events and this could be used as a reference in many managerial jobs like product manager and consultancy.


Together with Schroeter and Lit Soc, it forms the three independent "Big" trophies that hostels compete for every year. Tech Soc is being re-envisioned as a forum with a much wider reach than just inter-hostel events. It has now become the umbrella under which all tech-related student initiatives will be encouraged and nurtured. Apart from these, one can join various clubs related to the technical world in the Centre for innovation (CFi).

TechSoc allows you to get your hands dirty and build stuff on your own or participate in tech quizzes or watch the spectacle as the different hostels battle.



Scoring System:
NOTE: The participants' hostel will be the one they are currently residing in this semester. For example, for Cauvery residents, while virtual allocation might say something else, since they are residing in Cauvery for the July-November 2022 semester, they will be representing Cauvery.

The Inter Hostel Events of IIT Madras are divided into 4 categories - Low Prep, No Prep, Mid Prep, and High Prep:

Event Difficulty - Meaning
No Prep - No preparation required from participants' side. Event is on-spot and will be over within 2-3 hours.
Low Prep - 1-5 days of work required from participants.
Mid Prep - 6-12 days of work required from participants.
High Prep - 12+ days of work required from participants.

Points are awarded to any participating team from any hostel for 3 reasons - Winning Points (decided as per event difficulty), Parti Points, and Brownie Points.

NOTE: While the total points for each event category shall be retained, the points breakdown may change as per the event difficulty level. For example, in an event where submission itself is a difficult task, the variance between winning points awarded for different positions will change.

Parti Points - These points shall be awarded to any "parti"-cipating team/individual representing the hostel.

We have defined a "normalization factor" (N.F.) to ensure an equitable distribution of points as per hostel strength. This is done to ensure that a similar participation percent from different hostel strengths is awarded equally.

Normalization factor = Number of participants from hostel / Hostel strength

Total Participation Score = Normalization Factor * Event Difficulty Constant

For different levels of event difficulty, different constants will be set.

Brownie Points - These points shall be awarded for a team/individual that did not get a podium finish but displayed exemplary performance. This will be at the discretion of the organizing club. More than 1 team can be awarded different values of brownie points.

Motto of techsoc: Environment, Resources, Sharing.
Tagline of techsoc: What I cannot build, I do not understand.
Believes of techsoc: We fight. We build. We win. And eat pizza.

TECHNICAL SOCIETY, IIT MADRAS
Address: Sudha and Shankar Innovation Hub, Indian Institute of Technology, Madras.

CONTACT DETAILS:
TechSoc Heads:
Prabhat - +91 91082 59723
Anouksha - +91 94978 44857
techsoc@smail.iitm.ac.in
"""

Sangam = """
Sangam serves as the conglomeration of all cultural clubs at the Institute, making it one of the largest student-run teams in the Institute. Sangam houses within it 16 different clubs, each catering to a unique and a niche art form and audience. Together these clubs redefine the cultural landscape of IIT Madras.

Cultural Contingent:
The Cultural Contingent of IIT Madras comprises the most talented and skilled people in the various art forms we cater to. As a part of the contingent, you represent IIT Madras in fests all around India, as well as in Inter IIT.

Insti Radio:
Insti Radio is a newly launched initiative aimed at connecting the campus in a unique, never-done-before way. There will be programs and fun segments 24/7, so be sure to check it out!

CLUBS
Music Club:
The IITM Music Club conducts jam sessions, karaoke nights, open mics, etc., generates content along with institute musicians and helps to better the institute’s music scene. The club also conducts the music events in Saarang and organizes the institute’s music festival Raagabop every year.

Drama Club:
Everyone needs some drama in their life. Come be a part of one of the most happening clubs in insti! From Games After Dark (our initiator event) to Stage Design, to Scriptwriting, to Stagecoach (our flagship event), the Drama Club has space for all! Join us as we celebrate over 60 years of existence and be part of a club that has nurtured many who went on to be professional thespians!

Choreo Club:
Choreo Club is committed to uplift the dance culture in IITM. The Choreo Club conducts workshops catering to people belonging to different proficiency levels throughout the semester. The club believes dance is all about moving in joy and the better you learn how to move the more joyful you become.

Classical Arts Club:
The Classical Arts Club plays a pivotal role in promoting and supporting the classical arts within insti, be it the budding rasika or the consummate vidvan. Over the past year, the CAC has played host to artistes of renown like Nithyasree Mahadevan, Leela Samson, Charumathi Raghuraman, and many more. Its various educational and collaborative initiatives have been much lauded, as it strives to be the one-stop destination for all classical art forms, being as inclusive as possible. As a hub for all manifestations of the Classical arts, from appreciation to performance, its activities are a rich addition to the insti cultural scene.

Comedy Club:
Laugh, cry, spit, and suffer.
Make jokes on your would-be lover.
We present The comedy club, boi.
To mediocrity, say goodbye.
With events like stand-up comedy, couch potatoes, open mics, parody songs, the Comedy Club is sure to leave you in splits!

Oratory Club:
Want to engage in passionate wars of words? Want to voice out and justify radical stances? Want to move audience after audience with your eloquent and persuasive speeches? Or charm them with your brevity, wit, and humor? Fret not, for the Oratory Club is just the place!

Word Games Club:
Do you flip to the last page of the newspaper every day just to get to the puzzle section? Ever noticed that ‘the Morse code’ rearranges to ‘here come dots’? Or ‘New York Times’ to ‘monkeys write’? Whether you’re a Jack of all wordplay, a master of puns, a compulsive anagrammer, or just someone who finds the antecedent verbosity even remotely relatable, then this is the place to be!

Writing Club:
Have you always been a bookworm, flipping through one book to another with ease? Are you a connoisseur of words, weaving poetry and tales with ease? Are you someone bursting with stories to tell but don’t know where to start? The Writing Club is a community for everyone, ranging from novices to professionals, poets to artists, and readers to general enthu nuggets who want to explore all insti clubs!

Quiz Club:
One of the oldest clubs on campus, the IITM Quiz Club is one of the best in the country. If trivia excites you, if you like to tickle your grey cells by asking and answering questions, if you want to enjoy the thrill of “cracking” a well-framed question, then QC is the place to be!

Media Club:
We are a group of enthusiastic people passionate about photography, film-making. If you are someone who finds capturing moments, telling stories, and showing the world around through your perspective, then this is the place to be! We at the club enable you to pursue this art and have fun in the process through a super exciting array of workshops, lectures, photo walks with professionals to scenic places, and competitive events.

Fine Arts Club:
Fine Arts Club is a club that primarily focuses on inculcating the culture of art and its varied forms amongst the students through its events and workshops. We believe that "Art makes you lose yourself and find yourself at the same time."

Design & VFX Club:
Weaving technology with creativity, the Design and VFX Club focuses on the actualization of creativity and digital media and tools. Hop on to our wagon to explore digital illustration, graphic design, animation, UI/UX, video editing, and loads of other cool stuff! (And discover the opportunities in those fields en route xD)

Fashion Club:
Interested in Paris fashion week? Avid fashion lovers, here's Fashion Club for you. The club gives you an opportunity to be a part of ramp walking workshops, modeling, and other aspects of fashion. Apart from this, the club also organizes Mr. and Miss Freshie, Unfold, and Panache, one of the biggest fashion shows in India.

Culinary Arts Club:
Culinary Arts Club is the newest addition to the Sangam family! If you're fascinated by Masterchef and love food, Culinary Arts Club is the place to be! Help steer insti's food and cooking culture through workshops, competitions, food fests, and more!

Informals Club:
Informals Club endeavors to bring together insti through a variety of fun and entertaining events throughout the year. In a better world, you’d see numerous life-sized games at KV grounds, the happy faces of people playing amusing games near the Carnival, and the intense Street sports games happening around during Saarang, all thanks to us.

eSports Club:
Want to make your way in the fastest-growing industry of the digital era? The eSports club is the place to be – from streamers to professionals, we have it all. Be a part of the largest eSports event in South India or just help your hostel survive in the midst of fierce inter-hostel rivalries. Watch the best of the best compete or be one of them, one of the youngest and fastest-growing clubs of the insti welcomes you.

Nova:
Ever thought of being part of the biggest cosplay event in Chennai? Ever thought which team brings up all the new events of Saarang? The answer is Nova. If you want to be a part of the team which brings up something new, exciting, and quirky every year, join the ever-evolving team of Saarang, Nova.

Spotlight:
Spotlight is the Saarang team which takes care of all the lecture series. From GVM to the cast of Scam 1992, Spotlight 2021 was power-packed with individuals from all walks of life. Previous speakers include Kamal Hasan, Nawazuddin Siddiqui, Shashi Tharoor, Sid Sriram, and many more. By the end of Saarang, your insta is going to be filled with your photos with celebrities.

Meraki:
Meraki aims to bring you the unconventional at every Saarang. In 2020, we had the Ferris wheel and the Columbus ride, the various inflatables at KV and the Vintage Car Expo. In 2021, you would have been intrigued by the Tatting workshop and amazed by the Whistling act, Carnatic fusion act, and the sand art act. In short, Meraki brings life to the streets of Saarang and bling to your Saarang experience.
"""

E_cell = """

Within the lush green premises of IIT M, the Entrepreneurship Cell was set up in the year 1998, with its former name being Centre for Technological Innovation, Development and Entrepreneurship Support (C-TIDES).

The Entrepreneurship Cell has developed as a cornerstone of the thriving startup culture at IIT Madras. The Insti startup culture is one long, captivating tale of change. E-Cell collaborates closely with the Centre for Innovation (CFI), the Pre-Incubation Cell - Nirmaan, and the Incubation Cell to make the entrepreneurial journey of individuals and startups more comfortable. By doing this, they are able to inspire the spirit of entrepreneurship among other nascent but anxious ideators. By planning conferences, social events, lectures, speaker engagements, and workshops, we hope to unleash the students' hidden creative potential.

Harbouring a population of over 10,000 people including students, faculty and general staff, it is reputed internationally for its quality of teaching and outstanding calibre of the students from its graduate and post graduate programs. Not only is the Institute pioneering with blossoming innovations and cutting-edge technology, but also the swarm of ventures and the impact that it has created and will be creating in the innovation and entrepreneurship ecosystem as a whole.
No no
Over the years, Entrepreneurship Cell is known to have nurture some of the best startups in the entrepreneurship arena like Detect Technologies (Forbes 30 Under 30 – Asia), Hyperverge and many more among others and had privilege to work under the leaders like Mr. Tarun Mehta (Co-Founder, Ather Energy) being the Head of the organization during his Institute Days and are.

Vision
We strive continuously towards nurturing individuals with creative minds, innovative ideas and potential for substantial business opportunities. We seek to enable smooth and efficient interaction between its principal components spanning students, faculty, working professionals, aspiring and existing entrepreneurs, mentors, angel investors and venture capitalists, thus consolidating the entire pipeline of entrepreneurship ecosystem and serving our vision to Empower Entrepreneurship at IIT Madras.

Mission
We, at The Entrepreneurship Cell, are missioned with a common goal to inculcate the spirit of entrepreneurship among young minds and become a pathway towards India’s socio-economic development, for India to emerge as the super-nation in the coming years. In its glorious 20 years of existence, E-Cell now created its unique brand value on a pan-India scale with a plethora of initiatives taken up and conducted along the line of our vision to empower entrepreneurship.

What E-cell does:
Entrepreneurship Cell IIT Madras makes the entrepreneurial world accessible to all group of people part of the IIT Madras ecosystem, cater the audience from startup enthusiast to hardcore entrepreneurs, with plethora of yearlong events like Conclaves, Keynotes, Industry-defined Problems, B-Planning Competitions and Workshops, through both personal and collaboration-based approach.

Some of our satellite activities include Startup Series Sessions, and our annual entrepreneurship fest, E-Summit - largest in the region and from this year is going to be the first student-run ISO Certified Entrepreneurship Event. E-Summit is now revamped to make it more focused and helpful for all the stakeholders, thus making our initiatives and support system accessible to not only our campus, but the community outside as well, through several dedicated programmes like Campus Director Programme and Entrepreneurship Cell Development Drive. We have now even started and are now the first in the domain to promote entrepreneurship in school students through initiatives like Education-21 (E-21) and nurture entrepreneurial mindset in them.

Have you ever questioned why college students quit lucrative positions with global corporations to pursue entrepreneurship instead? The campus of IIT Madras is currently seeing a trend encouraging entrepreneurs. Every person, including you, is capable of having an entrepreneurial mindset, in our opinion.

Contact us
Dhawal Shukla: +91 78983 07770
sr_ecell@smail.iitm.ac.in
"""

Sports = """
Sporting at IIT Madras is not merely an activity. It has, over the years, gained so much ground that it finds itself ingrained in the IITM culture. The level of enthusiasm this sporting culture generates extends beyond the campus walls; it's a source of an adrenaline rush for other colleges in the city and the country as well. The academic calendar is crowded with sporting events, including intra-hostel and inter-hostel events, as well as inter-collegiate and inter-IIT tournaments. All the hostels actively compete to win the coveted Schroeter Cup, which is the Inter-Hostel Sports Championship. In addition to all these, we've added a couple of surprises for you this year! The institute has excellent sporting facilities on the campus, which include:

1. K.S. Narayanan Center for Cricketing Excellence
2. A perfect Athletics stadium
3. Four synthetic-floored Tennis and wood-floored Badminton courts
4. An Olympic-size Swimming Pool
5. Hockey and Football fields with floodlights
6. Two Basketball and two Volleyball courts
7. A well-equipped Gymnasium with professional instructors
8. Two world-class Squash courts
9. Weightlifting Arena
10. Table Tennis facility
11. A skating ring around Sangam ground

The surprise of the year awaits you in the form of 'Sporting Clubs.' For all those passionate about sports, we are forming Sporting Clubs this year - a whole new initiative to give you an unparalleled premier sporting experience year-round! The currently planned Sporting Clubs are:

1. Aquatics Club
2. Athletics Club
3. Badminton Club
4. Bridge Club
5. Cricket Club
6. Football Club
7. Hockey Club
8. Squash Club
9. Table Tennis Club
10. Tennis Club
11. Volleyball Club
12. Weightlifting Club
13. Chess Club
14. Forest Gumps
15. Frisbee Club
16. Fitness Club
17. Cycling Club
"""

Electives = """
At IIT Madras, students have the freedom to select 50% of their course offerings. This includes the option of spending the entire semester studying abroad and pursuing electives from any department.

Even among the IITs, Indian Institute of Technology Madras boasts one of the greatest academic curriculum available and provides unparalleled academic flexibility. The Ministry of Education, Government of India, has given IIT Madras the remarkable distinction of being the only Indian Institute to have been ranked as the No. 1 Engineering Institution for five consecutive years.

In IIT Madras, students have the freedom to choose nearly 50 per cent of the courses that they will be studying, with only some restrictions. Out of the electives, students are free to choose about eight courses (called 'Free Electives') from any of the 16 departments of IIT Madras as well as other approved institutions. The students can get 72 credits from pursuing courses as diverse as 'History of India,' 'Psychology,' and 'Economics' to 'Artificial intelligence' and 'Machine Learning.'

How to choose courses in IITs?

If a student chooses four such courses in one contiguous area, they will be conferred a minor in that field. For example, if a student chooses four courses in Economics, IIT Madras will confer a Minor in Economics to that student. There are large numbers of minor courses being offered by IIT Madras.

Honours_programme
To obtain an honours degree, students will have to maintain a CPA greater than or equal to 8.5, complete a 27-credit project (wherein one is expected to put in 27 hours of work every week throughout the 16-week semester) and earn at least 18 of the free elective credits in their parent department. The program, however, is open to only BTech and Dual Degree students.
"""

Student_governance = """
In India, there are very few colleges that offer a supportive environment where students can express their concerns and thoughts. But IIT Madras is distinct from other universities. The respect accorded to student voices, which has been codified in the form of the Students Constitution, is one distinct feature of our institute that distinguishes it from other colleges. An outline of student governance in general, including its elements and roles, is provided here.

Student Legislative Council & Executive Council

The student legislative council is also tasked with assisting the offices of the deans of students, academic courses, academic research, and international & alumni affairs in running day-to-day student matters through the Executive council. The Student Governance process of our Institute is most prominently represented by the Executive Wing, which is made up of student representatives who are directly elected by the General Student Body.
1. The Student General Secretary
2. The Hostel Affairs Secretary
3. The Sports Secretary
4. The Co-curricular Affairs Secretary
5. The Academic Affairs Secretary
6. The Research Affairs Secretary
7. The International and Alumni Affairs Secretary
8. The Cultural Affairs Secretary (Literary)
9. The Cultural Affairs Secretary (Arts)
10. Speaker, Student Legislative Committee

The Student Legislative Council elects a Speaker who preside over meetings, oversees the day-to-day operations of the Student Legislative Council, and serves as a liaison between the Student Legislative Council and the Executive Wing in addition to these nine secretary members.
"""

Indep_bodies = """
SABHAS AND SAMITIS
There are a lot of regional clubs on campus that organize cultural events, get-togethers, and meals throughout the year. Regional club members include not only students, but also faculty and staff. In the first year, they are a great way to make new friends and keep homesickness at bay, so watch out for notices on campus announcing their events, especially for freshers. The regional clubs on campus include:

Hindi Mitra Mandal (HMM)
Hindi Mitra Mandal is a group of enthusiastic students who celebrate the diversity of our country. We have a good history of organizing Festivals, cultural programs, and joyful events like Holi Celebration, Krishna Janmashtami, Diwali, and Navratri in our institute.

MARATHI MITRA MANDAL
Marathi Mitra Mandal is an Institute-recognized, a student-driven body coming under the various regional Samitis on the campus. There are more than 300 students coming from all over Maharashtra for various programs, and the organization aims at keeping the Marathi culture and spirit alive on the campus through various events and festivals, celebrated with great pomp in Maharashtra. Notable events conducted include Ganesh Chaturthi, Dahi Handi, a musical event Indradhanu in the odd semester, and a drama event Snehabandha in the even semester.

GARVI GUJARATI SANGATHAN
Garvi Gujarati Sangathan is a body of Gujarati students on our campus. It has been formed with a view to provide a platform for Gujarati students on the campus to know, meet, and bond with each other. GGS also aims to celebrate the cultural festivals which are prominent in Gujarat like Navratri, Shivratri, Janmashtami, and more, giving us a chance to enjoy and cherish the festivals and culture here away from home.

SADINER (Students Association for Development of India's North East Region)
SADINER is the association of the students coming from the North East region of India which is involved in various social and cultural activities in the campus. Our association has had an active participation in events like Independence Day, Republic Day, etc in the institute as well as played an active role in social activities like organizing flood relief funds for the North Eastern states. Our flagship event is the North East India fest which is organized every year in the Spring Semester.

DIGANTO
Diganto, the Bengali Association of IIT Madras is not only an organization that unites all Bengalis who are part of this institute, but it is more like a family where we meet people speaking our native language and share the same culture that brings us the essence of being at home in Chennai.

KARKA KASADARA TAMIL CLUB
The objectives of the club provide and maintain social, academic, and recreational amenities for its Tamil members and to teach Tamil to non-natives, to acquaint them with the language and its culture as well as to conduct training and workshops by teachers of foreign languages and Tamil, to both the non-native and the native audience.

TELUGU SAMSKRITIKA SAMITHI
Telugu Samskritika Samithi is a group of enthusiastic students who celebrate the Telugu cultural diversity of our country. They organize Festivals, cultural programs, and celebrate events like Vinayaka Chathurthi, Krishna Janmashtami, Rama Navami in our institute.

KERALA KALA SAMITHI (KKS)
KKS was initiated decades ago and grown with the support of faculty and students. The precious legacies, powerful aspects of language, poetry, drama, dance, Mohiniattam, Kathakali, and so on chiefly based in Kerala, are really fascinating and it will be of immense individual and collective advantage to arrange for sharing the experience and insights of the population from one region with those of the neighbors, structured on a sober platform of long-cherished ethic through this platform.
"""

WiFi_LAN = """
LAN is available in every room of all Hostels. Students have to bring their own LAN cable to connect it to their Laptop/PC. And Wi-Fi is available in Academic Section, In Library and many more places.

For connecting to Wi-Fi, you just have to select iitmwifi in the Wi-Fi option on your device and connect it through your roll number and LDAP Password.

General Instruction to connect LAN:

Firstly, we have to do Remote Proxy Setup.
Remote Proxy Setup:
If you require access to workflow.iitm.ac.in, academic.iitm.ac.in, or other internal websites OR journals available through IITM subscription, other restricted websites available only through IITM network and do not otherwise need access to licenses or servers inside IIT Madras, please use the following proxy setup instead of VPN. Instructions are given here for Firefox only - for Chrome or Internet Explorer/Edge, you will need to see how to set up the proxy options. Also, the instructions here are only for the desktop, so please give feedback if you know how to make it work for mobile browsers.

Step 1: Open the “Preferences” page in Firefox.
Step 2: Open the “Network Settings” page under the Preferences (search for “proxy” should do it).
Step 3: Set the Proxy Auto-Config URL and reload if necessary.
Step 4: Log in with LDAP credentials (not ADS).

Effect:
Once the above steps are done, you should find that only traffic destined for workflow, academic, journals, etc., goes through the proxy, and everything else goes through your regular network. For any issues, mail to helpdeskcc@iitm.ac.in.
"""

Websites = """
courses@iitm.ac.in:
This is the main academic website that gives you a list of all your courses and the teachers that teach you those courses. It is the all-in-one academic platform where you’ll be given assignments and additional course material. All your quiz papers, tutorials, and solutions will be uploaded on your personal account. In fact, some professors even prefer taking quizzes through this. The website has an app version called Moodle which is a must-download. You can log in to Moodle using your L-DAP credentials, which is basically your roll number as your login ID and a unique password. ‘Uploading on Moodle’ is a term that you’re going to hear very often throughout your insti life.

workflow@iitm.ac.in:
Workflow IITM is the website that you’ll be using to register for all your courses officially. During the add-and-drop courses week, you can register and deregister courses if approved by your faculty advisor. Workflow can only be accessed by logging into netaccess.iitm.ac.in through your L-DAP credentials or using institute WiFi.

http://ccw.iitm.ac.in/:
This is the Hostel Management Portal of IITM. Any hostel-related issues, be it mess biometric registration, hostel fee payment, complaints, or suggestions, are to be addressed through this portal.

http://academic.iitm.ac.in/:
This website lists all the courses offered by different professors of the institute. It provides information about every course in detail. Again, accessible only through insti-net, it gives the students an overview of their academic curriculum.

https://nptel.ac.in/:
NPTEL or National Program on Technology Enhanced Learning is a website that has a collection of educational videos on various courses taught in the curriculum of the top 7 IITs and IISc. These courses are taught by the professors themselves and are accessible to everybody. You can enroll and get certified for the same.
"""

NSO_NCA_NSS_NCC = """
Talking to freshies, I am reminded of the same awe and bewilderment with which I navigated my first few weeks of insti life. And naturally, one of the biggest conundrums we faced back then was choosing one of NSO, NSS, NCC, or NCA. This article hopes to throw some light on the specifics of each so as to aid freshies in their decision-making process.

NCC (National Cadet Corps):

This is the most underrated of the available options owing to the various myths that cast it in a strict, cruel, and monotonous light. No surprise there, since NCC is known for its parade drills and working hours that will make you wake up at dawn-break. But the battalion in insti happens to be an air force tech squadron, which means that there is less focus on drill and more on other technical and practical aspects of flying. In fact, drill practices are held only for stretches of about a week right before the Independence and Republic Day celebrations. In the rest of the meetings that are held every week, the cadets are given fitness training and might indulge in a spectrum of activities from learning to fly drones to even watching movies about flying. These sessions usually last a couple of hours starting at 6 in the morning.

NSS (National Service Scheme):

NSS is an excellent opportunity for students to step out of the comfort of their everyday life and engage in social service. With its wide range of projects targeting different sections of the population, NSS has over the years made significant contributions to the society. Its projects can be broadly classified into teaching projects, awareness projects, and content generation projects.

NSO (National Sports Organization):

NSO is the most popular option among students. No surprise there. Don’t we all love a good game? NSO offers the option of choosing from a variety of sports spanning cricket, athletics, frisbee, yoga, weightlifting, aquatics to name a few. The entire list can be found in the smail regarding the same.

NCA (National Cultural Appreciation):

This is the newest addition to the list, having started the year before. This is a great opportunity for students to hone their skill in a cultural activity of their choice or even learn an entirely new skill. Roughly 100-120 selections are in place.

Now that you have a fairly good idea of each of NSO, NCC, NSS, and NCA, I would like to take this opportunity to address another question frequently asked by most freshies.

You don’t have to worry about being left without being selected in anything. There are exactly as many vacancies as there are freshmen.

So, at the end, all those students who’ve been selected in more than one of NSO, NCC, etc. will have to choose exactly one and drop the rest. The vacancies will be filled by those students who haven’t gotten selected in anything yet.

At the end of the day, no matter what you take, just remember to slay it.
"""

DostE= """

DostE is a sophisticated AI chatbot tailored exclusively for college students, designed to assist them in acquiring essential information about basically everything and anything related to college.
It is initially crafted as a tech project for BuildSchool 3.0 by three passionate IIT Madras students- Kanishk Kala, Charchit Paharia and Nikshay Jain in their third year, has transformed into an indispensable tool for every IIT Madras student. Despite our limited tech background, we've built dostE into a go-to resource, providing essential info on courses, clubs, and everything in between. Now, it's the go-to hub for all students at IIT Madras! dostE database is vast, covering every tiny detail imaginable – from the number of cats on campus to bustling hours, mess schedules, and even the whereabouts of the elusive third elephant of GC.

The dataset of dostE contains almost everything related to the institute and going above and beyond that. After buildschool dostE has a high potential to be incubated in the nirmaan incubator to go on becoming the first ai chatbot startup built entirely by students of iit madras. In forming dostE a good amount of help and assistance was needed from alumni like Ankit Sanghvi, and students like Jayanth, Archish , Sudip and Abhishek.
"""

MA1101 = """
Course name: functions of several variables
Course code: MA1101

This is a 1st year course common for all the students across the btech curriculum. The course gradings are typically RG, this course takes place in a series of sections with all the 1st year students divided in different sections. This course along with the others play a significant role in the decision of opting for a branch change. Usually the course grading percentage is 25% quiz 1, 25% quiz 2, and 50% endsems with more importance laid on the in class and homework tutorials. Class usually takes place in RMN or CRC but it varies with the course instructor. Course instructors are different in different sections. Mainly the course instructors for this course are:

asingh1 Arindama Singh
 balaji51 Balaji R
 dipramit1 Dipramit Majumdar
 kmahalingam1 Kalpana M
 radharam1 Radha R
 rameshk1 Ramesh Kasilingam
 Sarang Sane
 sjroy1 Satyajit Roy
 bsriram1 Sriram B

The course details are:
Functions of two and three variables: Regions in plane, level curves and level surfaces, limit, continuity, partial derivatives, directional derivatives and gradient, normal to level curves and tangents, extreme values, Lagrange multipliers, double integral and iterated integral, volume of solids of revolution, approximation of volume, triple integral, change of variables, multiple integrals in cylindrical and spherical coordinates. Vector calculus: Gradient, Divergence, Curl, Line integral, conservative fields, Green's theorem, surface area of solids of revolution, surface area, surface integral, Triple integrals and Gauss Divergence theorem, Stokes' theorem.

Textbook for references are:
1.E. Kreyszig, Advanced Engineering Mathematics, 10th Ed., John Willey & Sons, 2010.
2. N. Piskunov, Differential and Integral Calculus Vol. 1-2, Mir Publishers, 1974.

Senior reviews about this course:
 Course is pretty lite- this course is easy if you study for an hour everyday. The course grading is easy, especially Sriram sir gives good grades. Along with that more stress could be laid upon vector and multiple integrals as the questions asked in the endsems are majorly from these portion. I would suggest everyone to watch youtube videos for these topics to understand better. As this a level 1 course it is really hard to fail the course unless you did really bad, so try to get good grades by studying. Also ensure to go to class regularly to inculcate in yourself the good qualities of regularity and determination from an early stage. This would certainly help you gain an upper edge.
This course offers a thorough exploration of multivariable calculus, emphasizing key concepts like limits, partial derivatives, and integrals. The focus on real-world applications, especially in optimization and vector calculus, equips students for advanced coursework and practical problem-solving. The inclusion of theorems like Green's and Stokes' enhances the theoretical depth, making it an excellent precursor for further studies in mathematics and physics.
The course adeptly blends theory with practical applications, providing a solid understanding of functions in two and three variables. The emphasis on directional derivatives, gradients, and optimization techniques prepares students for real-world problem-solving. Covering double and triple integrals adds depth, making it a valuable stepping stone for advanced calculus and related disciplines.
This course navigates through multivariable functions, laying a strong foundation with an emphasis on limits, continuity, and partial derivatives. The practical focus on optimization, double integrals, and vector calculus bridges theory to real-world scenarios, making it an essential precursor for advanced calculus and physics. The inclusion of theorems like Gauss Divergence and Stokes' broadens the horizon for interdisciplinary applications.
"""

MA1102 = """

Course name: Series and matrices
Course code: MA1102
Course grading: RG

This course is offered in the 2nd semester to all the btech students at IIT Madras, usually happens in RMN block and is offered in the C slot. This course follows a relative grading. Course is broken down into 2 quizzes and 1 endsemm with each quiz having 25% weightage and endsems having 50% weightage. This course breakdown is subjected to change depending upon the professors. Professors who usually take this course are Arindama Singh, Jayanthan AV, Kulkarni SH, Neelesh S Upadhaye and Jaikrishnan.


Course Details:
Series: Sequences of real numbers, Series, ratio and root test, improper integral, integral test, alternating series, absolute and conditional convergence, power series, radius and interval of convergence of power series, term by term differentiation and integration of power series, Taylor?s formulas, Taylor series, periodic functions and Fourier series, convergence of Fourier series, functions of any period, even and odd functions, half-range expansions.
Matrices: Matrix operations, special types of matrices, matrices as linear transformations, linear independence, basis and dimension, rank of a matrix, nullity of a matrix, elementary operations, inverse of a matrix, orthogonalization, determinant, existence- uniqueness of solutions of a linear system, Gaussian elimination, Gauss-Jordan elimination, Eigenvalues, eigenvectors, eigenvalues of special types of matrices, similarity of matrices, basis of eigenvectors, diagonalization.

Course References:
1. G.B. Thomas Jr., M.D. Weir and J.R. Hass, Thomas Calculus, Pearson Education, 2009.
2. E. Kreyszig, Advanced Engineering Mathematics, 10th Ed., John Willey & Sons, 2010.
REFERENCES:

Senior reviews:






This course is relatively easy course as compared to MA1101, the course grading is easy, people dont usually fail in this course. Even if you have not studied anything questions could be asked from multiorder differentiation, diagonalization and taylor’s formulae which could fetch you some marks as throughout the years these topics were asked in the endsem and since people didnt prepared these topics well for various reasons these easy topics downgraded the grades of people as well.

This course delves into the intricacies of sequences and series, covering convergence tests, power series, and Fourier series. The practical applications, such as term-by-term differentiation and integration, offer insights into real-world problem-solving. The matrices section, encompassing operations, eigenvalues, and diagonalization, provides a solid foundation in linear algebra. The course's comprehensive nature equips students with the tools to navigate through the complexities of mathematical analysis and matrix algebra, setting the stage for advanced coursework.

"Series and Matrices" is a comprehensive course that masterfully navigates through the realm of mathematical analysis and linear algebra. The detailed exploration of series, from convergence tests to power series applications, builds a robust understanding of sequences. The matrix section covers a wide array of topics, including linear transformations, eigenvalues, and diagonalization, providing students with a solid grasp of fundamental linear algebra concepts. This course serves as a vital bridge to advanced mathematical studies, offering a rich blend of theory and practical applications.

The "Series and Matrices" course serves as a fundamental cornerstone in mathematical analysis. The exploration of series, including convergence tests and power series, provides a deep understanding of sequences and their behavior. The application of Taylor's formulas and Fourier series adds a practical dimension, enabling students to tackle real-world problems. The matrices section covers essential operations, linear transformations, and eigenvalues, laying the groundwork for advanced linear algebra. The course's structured approach makes it an indispensable prerequisite for further studies in both pure and applied mathematics.
"""

PH1020 = """

Course name: Physics ii
Course code: PH1020
Course grading: Relative grading


This course is offered in the 2nd semester common to all btech students. Course grading is relative and this course is taken up by a group of professors some of them being Ashwin Joy, Aravind G, Basudev Roy, Dawood Kothawala and Dilip Kumar Satapathy. It generally is alloted in the A slot of the semester with midsem having 30 % marks, tutorials containing 20% marks and rest 50% marks are for endsems.


Course content:

Unit 1-The electromagnetic force as a fundamental force, the electrostatic field as a conservative vector field: basic features, Principle of superposition, discrete and continuous charge distributions. The concept offiel
d lines and flux, Gauss’s law, Examples of finding the field using Gauss’s law.
Unit 2-The electrostatic potential, differential form of Gauss's Law, Poisson's equation, Electrostatic energy of acharge distribution, Equilibrium in electrostatics
Unit 3-Charged surfaces, conductors and capacitance, capacitors. The idea of multi pole expansion, Monopoleand dipole moments, contributions to potential.
Unit 4- Dielectric media: effect of an electric field on a dielectric, electric displacement vector, polarization vector, Linear-Isotropic-Homogeneous media, Boundary conditions, Electric field in a dielectric, energydensity.
Unit 5-Magnetostatics: Magnetic induction, force on a current carrying conductor in a magnetic field, Biot-Savart’s law. Ampere's Law, Magnetic field due to a current.
Unit 6-Electrodynamics: Force on a charged particle in electromagnetic fields. Time-varying fields. Faraday'slaw of electromagnetic induction. Self and mutual inductance. Maxwell's equations in free space.
Unit 7-Electric displacement vector, dielectric Susceptibility. Boundary conditions. Magnetic induction,Magnetization. Energy density in a magnetic field. Magnetic permeability and susceptibility.
Unit 8- Maxwell’s equations in a dielectric medium. Electromagnetic waves, Monochromatic plane wave propagation in free space. Wave propagation in dielectric media, energy density. Electromagnetic energydensity, Poynting vector.

Some reference books for these are:
Introduction to Electrodynamics" by David J. Griffiths
Electricity and Magnetism" by Edward M. Purcell

Reviews by seniors:

A course a bit on the tough side. Average grades people usually get is C. It is very difficult to get an S grade. You need to constantly and regularly study. Average score people get in the midsems and quizzes are low which tells that it is hard for students to grasp the concepts being taught in class. I would suggest to go through youtube video lectures especially from Physics academy and alura to understand the concepts better. Try to pay maximum attention in the class and dont skip classes as it would be hard to understand the topics you missed. Major chunk of portion comes from maxwell’s equation and applications, gauss’s law and magnetostatics
Physics II (PH1020) offers a comprehensive study of electromagnetism, covering foundational concepts such as the electrostatic field, Gauss's law, and the electrostatic potential. The course delves into advanced topics like dielectric media, magnetostatics, and electrodynamics, providing a deep understanding of Maxwell's equations and their applications. The inclusion of practical examples and the exploration of electromagnetic waves make this course an essential stepping stone for students pursuing further studies in physics or related fields.

PH1020 presents a detailed examination of electromagnetic forces, focusing on the electrostatic field and its conservative nature. The course skillfully progresses through topics like Gauss's law, capacitance, magnetostatics, and electrodynamics. Noteworthy are the discussions on dielectric media and Maxwell's equations, providing students with a solid foundation in the understanding of electric and magnetic fields. The integration of theoretical principles with real-world applications makes this course indispensable for students seeking a robust grasp of electromagnetism.

Physics II (PH1020) meticulously guides students through the intricacies of electromagnetism, starting with fundamental principles like electrostatics and Gauss's law. The exploration of capacitance, magnetostatics, and electrodynamics enriches the understanding of complex electromagnetic phenomena. The inclusion of dielectric media and Maxwell's equations adds depth to the theoretical framework, preparing students for the practical application of electromagnetic principles. This course serves as a crucial bridge between foundational physics and advanced studies, offering a well-rounded exploration of both theory and application.
"""

PH1010= """

Course name: Physics 1
Course code: PH1020
Course grading: Relative grading

This is a 1st year course common for all the students across the btech curriculum. The course gradings are typically RG, this course takes place in a series of sections with all the 1st year students divided in different sections. This course along with the others play a significant role in the decision of opting for a branch change. Usually the course grading percentage is 20 % quiz 1, 20% quiz 2, 10% tutorials and 50 % endsems but it may also depend on the professors. Classes have been happening in RMN block only and this course is usually offered in the A slot.
Generally this course is taken up by prof Aravind G.,

Course format: The tentative format of the end-of-semester exam last year was as follows. The exam  contained 4 true/false questions (for 1 mark each), 4 multiple choice questions with one or more correct options (for 2 marks each), 8 fill in the blanks (for 1 mark each), five questions involving moderate calculations (for 3 marks each) and three question involving detailed calculations or plotting (for 5 marks
each), adding to a total of 50 marks.
"""

CS1100 = """

Course name: Introduction to programming
Course code: CS1100
Course grading: Absolute grading

This course is offered in 2nd semester to majorly all the departments except CS. ED and BT. This course is offered in the E slot and usually follows an AG grading system. This course usually have quiz 1 carrying 20%, quiz 2 carrying 20 % , the programming lab (replit) where you are given coding assignments and you have to submit assignments each week (30%) and endsems are 30%. The questions in quizzes and endsems are mainly objective and fill in the blanks. This course is often taken up by a group of 5-6 professors with each professor having different section of students. Professors are: Shweta Agarwal, Ananth Krishnan, Harish Gurukumar

Course syllabus:
0. Introduction to Computing, including basics of Computer
Organization (CPU, ALU, etc.), Machine Lang/Assembly/High-level
languages, and program compilation steps.

1. Data representation (Decimal, Binary) of signed and unsigned
integers (Sign-Magnitude, One's and Twoâ€™s complement), floating point
numbers.

2. Variables and Constants, Assignment Statements, Operators --
Arithmetic, Logical, Relational, operator precedence).

3. Input/output: Format-strings for printf and scanf.

4. Conditional statements: if-else, switch-case

5. Repetitive statements: while, do-while and for loops

6. Arrays: one dimensional and two-dimensional arrays

7. Char data type, Strings and String Manipulation functions

8. Functions and Modular Programming

9. Recursive Functions

10. Structures

11. Pointers

12. Searching algorithms: Linear and Binary search, where elements are
stored in an array.

13. Sorting algorithms: Selection and Insertion Sort, where elements
are stored in an array.

References:
Paul Deitel and Harvey Deitel. C: How to Program.
V. Rajaraman: Computer Programming in C.
R. G. Dromey: How to Solve It By Computer?
Kernighan and Ritchie: The C Programming Language.

Reviews by senior students:
This course is a very easy course if you know the topics well and can do programming anf a very hard course if you have no interest in programming and this can be deadly. People often fail this course because they take this course lightly. The questions asked are multiple choice questions with one correct answer and the grading is pretty easy. It is easy to score full also if you know your concepts well. No extra material required as such, no reference books, going to class can be an option. You can watch code with harry videos or shraddha kaphra videos for this course. Aman Dattarwal’s 12 hour one shot video in C programming is pretty good for this course.
This introductory programming course provides a solid foundation for students entering the world of computing. It covers essential concepts such as data representation, variables, operators, and input/output handling. The emphasis on conditional and repetitive statements, along with the introduction to arrays, lays the groundwork for algorithmic thinking. The inclusion of functions, recursion, and structures adds depth to modular programming, making it an excellent starting point for beginners in the field of computer science.
This programming course effectively navigates through the fundamentals, offering a comprehensive introduction to variables, data representation, and basic control flow structures. The coverage of arrays, strings, functions, and recursive programming provides a well-rounded understanding of essential programming constructs. The incorporation of searching and sorting algorithms enhances problem-solving skills, making this course a valuable stepping stone for students aspiring to develop proficiency in coding.
The "Introduction to Computing" course is a pragmatic journey through programming essentials. Starting with the basics of computer organization and data representation, it systematically progresses through variables, control structures, and arrays. The emphasis on functions, recursion, and pointers enriches students' understanding of modular programming. The inclusion of searching and sorting algorithms introduces algorithmic thinking, making this course a practical and well-rounded introduction for anyone venturing into the realm of programming.
"""

EE1100 ="""

Course name: Basic electrical engineering

This course is offered in the third semester of chemical, mechanical, electrical, computer science and civil department. This course usually takes place in CRC 101, 102, 103 and is alloted in the E slot of the semester. The course is broken down into- 3 mini-quizzes of 5 marks each; Quiz1 : 15 marks; Quiz2: 20 marks; Final: 50 marks

Books to refer:  Electrical Circuit Theory and Technology, John Bird, Elsevier, 2011 [Text book for topics 1 to 4.  Essentials of Electrical and Computer Engineering, Kerns & Irwin, Pearson, 2004.

Course content: Introduction + Topic 1 :- Basics and definitions: definition of charge, voltage, current, power, energy, resistor, Ohm's law, ideal sources, internal resistance of voltage and current sources

Topic 2 :- Circuit laws and solving for unknowns: description & derivation of Kirchoff's laws, series and parallel connections, analysis of circuits using Nodal analysis technique usage of the principle of linearity and superposition to solve when multiple sources are present.

Topic 3 :- Dynamic circuit components L & C:derivation of the relationship between current
and voltage in capacitors(C) and inductors(L), power in L & C, combination of R & L , R & C, numerical & graphical analysis of transients with DC sources, combination of R, L & C in
circuits.

Topic 4 :- Single phase AC circuits :
Introduction to AC signals and sources, description of AC signals using magnitude, frequency and phase, distinguishing between peak, average and RMS values, definition of power in AC circuits, derivation of properties of R, L and C with AC signals, introduction to Impedance(Z), Differentiate between DC and steady state, calculation of power in AC systems, differentiating between active, reactive and apparent power, introduction to power factor, usage of phasors to represent, manipulate and calculate circuit quantities like voltage,current, impedance, and power.

Topic 5 :- How does the wireline telephone system work?
Local loop, analog to digital conversion, sampling of speech, compression of speech, echocontrol, blocking and non-blocking switches, application of Erlang-B formula to voice
networks, and how to send data thro the (voice-based) telephone network

Topic 6 :- Digital communications
Analog communications vs Digital communication, examples from source compression, errorcontrol coding, mapping bits to waveforms (transmit modulation), and recovering bits from noise corrupted waveforms (demodulation and detection at the receiver)

Topic 7 :- Packet Switching – advantage over circuit switching, how coding and packet
repetition can ensure “error-free” data transport over the internet

Topic 8:- Wireless transmission – transmit power, path loss, receiver sensitivity, and link
budget, point-to-point wireless links, wireless broadcasting.

Topic 9 (if time permits):- Mobile Cellular networks – evolution from voice-only (2G) to
mainly data applications

Course intructors for this course usually are: Prof Ananth Krishnan, Prof Anbarasu, Prof Amitava Das Gupta, Prof Abhishek Sinha

Senior reviews:
This course is as same as CS1100 course. If you study even a little you would get good grades and if you dont study at all you can get really bad grades or even fail this course. The quiz 1 for this course is the eassiest with simple general circuit analysis being asked. The average for quiz 1 get really high so to get good grades you have to go prepared for quiz 1. Quiz 2 is somewhat difficult than quiz 1 and therefore the average score drops. Mainly quiz 2 is the deciding factor for the grades. Endsems are relatively easy for this course. Average grades people get here is B. But getting a S grade is not at all difficult. This course requires practising and thorough understanding of the concepts. Make sure to attend the class regularly, if not atleast be aware what happened in the class you missed and watch it later on youtube.
The Basic Electrical Engineering course offers a solid foundation for students across multiple departments, introducing them to fundamental concepts through a structured curriculum. The inclusion of mini-quizzes and a well-distributed assessment scheme ensures continuous engagement. The course content, covering topics from circuit laws to dynamic components and AC circuits, is thoughtfully organized. The emphasis on real-world applications, such as the wireline telephone system and digital communications, enriches students' understanding and underscores the course's relevance in modern technology.
This course in Basic Electrical Engineering serves as a crucial bridge between theoretical principles and practical applications. The incorporation of Kirchhoff's laws, analysis techniques, and dynamic circuit components lays a strong foundation. The inclusion of topics like single-phase AC circuits and digital communications showcases the breadth of electrical engineering. The utilization of real-world examples, such as the wireline telephone system and wireless transmission, enhances the course's practicality. The choice of textbooks complements the course content, making it a valuable resource for students in various engineering disciplines.
"""



### Training the model

In [ ]:
# Chunck Documents
# Create Document object from text documents

docs = [Document(page_content=post) for post in [Intro, TechSoc, Sangam, E_cell, Sports, Student_governance, Indep_bodies, WiFi_LAN, Websites, NSO_NCA_NSS_NCC, DostE, MA1101, MA1102,  PH1020, PH1010, CS1100, EE1100]]

# Split documents into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=10, separators=['\n\n', '\n', '.']
)

document_chunks = text_splitter.split_documents(docs)

In [ ]:
#Initiate a Vector store Instance
# Initiate a chromadb instance

chroma_db = Chroma.from_documents(document_chunks, embedding_model)
retriever = chroma_db.as_retriever()

In [ ]:
# Define an LLM here

llm = mistral_llm

In [ ]:
#option 2
# Prompt template
qa_template = """<s>[INST] You are a guiding, helpful and honest assistant. Answer exactly in few words from the context
Answer the question below from context below :
{context}
</s>
[INST] {question} [/INST]
"""

# Create a prompt instance
QA_PROMPT = PromptTemplate.from_template(qa_template, additional_fields=['question', 'context'])


# Custom QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_PROMPT}
    )

In [ ]:
question_pp = "Tell me about the course CS1100? How to prepare for it"
response = qa_chain({"query": question_pp ,"context":[Intro, TechSoc, Sangam, E_cell, Sports, Student_governance, Indep_bodies, WiFi_LAN, Websites, NSO_NCA_NSS_NCC, DostE, MA1101, MA1102, PH1020, PH1010, CS1100, EE1100]})
print(response['result'])

## Integration with UI

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45227 sha256=6e7aed1a9ea7665200275cb3eee04c06b002f77397374ac52aa73472a3fafd4f
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


In [ ]:
import anvil.server
anvil.server.connect("server_IZQKCW3I6REZQ7DZTHXNBNFD-B4PJFSOXVF73P7WK")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as SERVER


In [ ]:
@anvil.server.callable
# Query Mistral 7B Instruct model

def askme(question_p):
  # context_p = """ On August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""
  # prompt = PromptTemplate(template=qa_template, input_variables=["question","context"])
  # llm_chain = LLMChain(prompt=prompt, llm=llm)
  # response = llm_chain.run({"question":question_p,"context":[test,Intro]})

  response = qa_chain({"query": question_p ,"context":[Intro, TechSoc, Sangam, E_cell, Sports, Student_governance, Indep_bodies, WiFi_LAN, Websites, NSO_NCA_NSS_NCC, DostE, MA1101, MA1102,  PH1020, PH1010, CS1100, EE1100]})
  return response['result']

In [ ]:
# anvil.server.wait_forever()

# ------------------------------------------------------------------END--------------------------------------------------------------------

## Model deploying

In [ ]:
import joblib
from google.colab import files

model = joblib.dump(llm, "dostE")
files.download(model)

PicklingError: ignored

In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load the pipeline
generator = pipeline('text-generation', model=model_name)

# Get the model associated with the pipeline
model = generator.model

# Save the model
model.save_pretrained('mistral_7b_model')

Deploy the backend using pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch

# Save your qa_chain model using torch.save
torch.save(model, "llm_model.pth")

In [ ]:
import pickle

# Save your qa_chain to a file
with open("dost_model.pkl", "wb") as file:
    pickle.dump(model, file)

# Define the desired path in Google Drive
desired_path = "/content/drive/MyDrive/dostE"  # Replace with your Google Drive folder path

# MAKE SURE YOU HAVE ALREADY MOUNTED YOUR DRIVE IN STEP 6

# Copy your model file from the current path in Colab to the desired path in Google Drive
!cp qa_chain_model.pkl {desired_path}

In [ ]:
# !pip install flask

In [ ]:
command = "pip install flask"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [ ]:
# !pip install colabcode
# !pip install fastapi

command = "pip install colabcode"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [ ]:

command = "pip install fastapi"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/', methods=['POST'])
def generate_response(r):
    # Get the input data from the POST request
    print(r)
    print(request.json)
    input_data = r.json
    return "hello"


from colabcode import ColabCode
cc = ColabCode(port=5000, code=False)
cc.run_app(app=app)


In [ ]:
app = FastAPI()

@app.get("/")
async def read_root():
  return {"message": "Subscribe to @1littlecoder"}

In [ ]:
cc.run_app(app=app)

In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pickle

iris = load_iris()
model = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
model_f = model.fit(X_train, y_train)

print("Model score: ", model.score(X_train, y_train))
print("Test Accuracy: ", model.score(X_test, y_test))

pickle.dump(model_f, open("model_gb.pkl", "wb"))

In [ ]:
%%writefile models.py
from pydantic import BaseModel, conlist
from typing import List

class Iris(BaseModel):
    data: List[conlist(float, min_items=4, max_items=4)]

In [ ]:
import pickle
import logging
from fastapi import FastAPI
from models import Iris

app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# # Initialize logging
# my_logger = logging.getLogger()
# my_logger.setLevel(logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG, filename='logs.log')

model = None

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_gb.pkl", "rb"))

@app.post("/api", tags=["prediction"])
async def get_predictions(iris: Iris):
    try:
        data = dict(iris)['data']
        print(data)
        iris_types = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }
        prediction = list(map(lambda x: iris_types[x], model.predict(data).tolist()))
        log_proba = model.predict_log_proba(data).tolist()
        return {"prediction": prediction, "log_proba": log_proba}
    except:
        my_logger.error("Something went wrong!")
        return {"prediction": "error"}

In [ ]:
cc.run_app(app=app)